In [45]:
import tensorflow as tf
import pandas as pd

In [46]:
train_dataset_fp = pd.read_csv('../iris_training.csv')
train_dataset = tf.data.Dataset.from_tensor_slices(
    (train_dataset_fp.iloc[:, :4].values, train_dataset_fp.iloc[:, 4].values))
train_dataset = train_dataset.batch(32)

In [47]:
# Model
class MyModel(tf.keras.Model):
    def __init__(self, num_classes=3):
        super(MyModel, self).__init__(name="MyModel")
        self.dense1 = tf.keras.layers.Dense(10, activation=tf.nn.relu)
        self.dense2 = tf.keras.layers.Dense(10, activation=tf.nn.relu)
        self.dense3 = tf.keras.layers.Dense(num_classes)

    @tf.function(input_signature=[tf.TensorSpec([None, 4], tf.float32, name='dcdmm')])
    def call(self, inputs):
        return self.dense3(self.dense2(self.dense1(inputs)))

In [48]:
model = MyModel(3)
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
optimizer = tf.keras.optimizers.SGD(learning_rate=0.01)


def loss(model, x, y, training):
    y_ = model(x, training=training)
    return loss_object(y_true=y, y_pred=y_)


def grad(model, inputs, targets):
    with tf.GradientTape() as tape:
        loss_value = loss(model, inputs, targets, training=True)
    return loss_value, tape.gradient(loss_value, model.trainable_variables)


num_epochs = 301

for epoch in range(num_epochs):
    epoch_loss_avg = tf.keras.metrics.Mean()
    for x, y in train_dataset:
        loss_value, grads = grad(model, x, y)  # 梯度计算
        optimizer.apply_gradients(zip(grads, model.trainable_variables))  # 执行一次优化步骤

        # 累计;不需要进行reset_state
        epoch_loss_avg.update_state(loss_value)  # Add current batch loss
    if epoch % 50 == 0:
        print("Epoch {:03d}: Loss: {:.3f}".format(epoch, epoch_loss_avg.result()))

Epoch 000: Loss: 1.990
Epoch 050: Loss: 0.630
Epoch 100: Loss: 0.404
Epoch 150: Loss: 0.288
Epoch 200: Loss: 0.210
Epoch 250: Loss: 0.164
Epoch 300: Loss: 0.138


In [49]:
predict_dataset = tf.convert_to_tensor([
    [5.1, 3.3, 1.7, 0.5, ],
    [5.9, 3.0, 4.2, 1.5, ],
    [6.9, 3.1, 5.4, 2.1]
])

model.predict(predict_dataset)

array([[  5.0435934 ,   0.62881607, -11.274467  ],
       [ -3.7962272 ,   1.0710709 ,  -2.2412043 ],
       [ -7.486438  ,   0.69437623,   1.4669781 ]], dtype=float32)

### 方法一:save_weights

In [50]:
model.save_weights('save_weights.h5')

In [51]:
model_k_h5 = MyModel(3)  # 重新创建网络
model_k_h5.build(input_shape=(None, 4))

In [52]:
model_k_h5.load_weights("save_weights.h5")

In [53]:
model_k_h5.predict(predict_dataset)

array([[  5.0435934 ,   0.62881607, -11.274467  ],
       [ -3.7962272 ,   1.0710709 ,  -2.2412043 ],
       [ -7.486438  ,   0.69437623,   1.4669781 ]], dtype=float32)

### 方法二:save

In [54]:
# ★★★★★只能保存为Tensorflow SavedModel,不能保存为HDF5 file
model.save('custom_model_tf_version', save_format='tf')

INFO:tensorflow:Assets written to: custom_model_tf_version\assets


In [55]:
new_model = tf.keras.models.load_model('custom_model_tf_version/')
new_model.predict(predict_dataset)

array([[  5.0435934 ,   0.62881607, -11.274467  ],
       [ -3.7962272 ,   1.0710709 ,  -2.2412043 ],
       [ -7.486438  ,   0.69437623,   1.4669781 ]], dtype=float32)

### 方法三:tf.savaed_model.save

In [56]:
# 一般用于模型部署
tf.saved_model.save(model, 'tf_saved_model_version')

INFO:tensorflow:Assets written to: tf_saved_model_version\assets


In [57]:
!saved_model_cli show --dir tf_saved_model_version --all


MetaGraphDef with tag-set: 'serve' contains the following SignatureDefs:

signature_def['__saved_model_init_op']:
  The given SavedModel SignatureDef contains the following input(s):
  The given SavedModel SignatureDef contains the following output(s):
    outputs['__saved_model_init_op'] tensor_info:
        dtype: DT_INVALID
        shape: unknown_rank
        name: NoOp
  Method name is: 

signature_def['serving_default']:
  The given SavedModel SignatureDef contains the following input(s):
    inputs['dcdmm'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1, 4)
        name: serving_default_dcdmm:0
  The given SavedModel SignatureDef contains the following output(s):
    outputs['output_1'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1, 3)
        name: StatefulPartitionedCall:0
  Method name is: tensorflow/serving/predict

Defined Functions:
  Function Name: '__call__'
    Option #1
      Callable with:
        Argument #1
          dcdmm: TensorSpec(shape=(None,

In [58]:
restored_saved_model = tf.saved_model.load('tf_saved_model_version')
f = restored_saved_model.signatures["serving_default"]
f(dcdmm=tf.constant(predict_dataset))

{'output_1': <tf.Tensor: shape=(3, 3), dtype=float32, numpy=
 array([[  5.0435934 ,   0.62881607, -11.274467  ],
        [ -3.7962272 ,   1.0710709 ,  -2.2412043 ],
        [ -7.486438  ,   0.69437623,   1.4669781 ]], dtype=float32)>}